In [ ]:
# ================================================================
# 03_kerr_flux_sim.ipynb  –  Kerr-enhanced quaternionic flux (fixed)
# ================================================================
# Colab cell 1 : 基本套件與常數
!pip install --quiet scipy

import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from math import sqrt, pi

# ---- 物理常數 (SI) ------------------------------------------------
G     = 6.67430e-11          # m³ kg⁻¹ s⁻²
c     = 2.99792458e8         # m s⁻¹
M_sun = 1.98847e30           # kg
M_pl  = 2.176434e-8          # kg  (Planck 質量)

# ---- 黑洞參數（Sgr A* 為例） --------------------------------------
M_bh   = 4.0e6 * M_sun       # 黑洞質量
a_star = 0.5                 # 無因次自旋  a* = J/(M r_g)  (0≤a*<1)

r_g_m  = 2*G*M_bh / c**2     # m
r_g    = 1.0                 # 把 r_g 當作長度單位 → r̂ 無因次
a_hat  = a_star              # 因為 r 已除以 r_g

print(f"r_g = {r_g_m/1e3:.2e} km,  a* = {a_star}")

# ---- 無因次 Δ 與其導數 --------------------------------------------
Δ  = lambda x: x**2 - x + a_hat**2
dΔ = lambda x: 2*x - 1

# ---- 耦合常數  λ̃(x)  (Appendix E) -------------------------------
kappa = 0.02
factor = (2*kappa) * (G*M_bh)/(M_pl**2 * r_g_m**3)  # SI → 1/m²
# 換成 1/r_g² (因 r_g=1) 並乘 r_g_m²，最後仍是無因次
λ_tilde = lambda x: factor / x**3 * r_g_m**2

# ---- ODE  --------------------------------------------------------
def rhs(x, y):
    ψ, χ = y                      # χ = dψ/dx
    dFdx = x**2 * λ_tilde(x) * ψ*(ψ**2 - 1)   # 右邊 source
    # F = x² Δ χ   ⇒  χ' = (dFdx - d/dx(x²Δ)·χ) / (x²Δ)
    dxd = (2*x*Δ(x) + x**2*dΔ(x))  # d/dx(x²Δ)
    χp  = (dFdx - dxd*χ) / (x**2 * Δ(x))
    return [χ, χp]

# ---- 穩定積分 ------------------------------------------------------
x_plus  = (1 + sqrt(1 - a_hat**2)) / 2      # r̂_+  (根)
x_start = x_plus * 1.02
x_end   = 100.0

y0 = [1e-6, 0.0]        # ψ≈0, ψ'≈0 at horizon
sol = solve_ivp(rhs, (x_start, x_end), y0,
                rtol=1e-10, atol=1e-13, max_step=0.1)

x_arr  = np.logspace(np.log10(x_start), np.log10(x_end), 600)
ψ_arr  = sol.sol(x_arr)[0]
ε_arr  = 2*ψ_arr

# ---- Analytic ansatz for comparison  -----------------------------
r_s0_hat = 8.5e3 * 3.085677e16 / r_g_m   # 8.5 kpc → r̂
boost    = 1.0 / (1 - a_hat / x_arr)
ε_ansatz = 2*np.tanh(x_arr / (r_s0_hat * boost))

# ---- Plot --------------------------------------------------------
plt.figure(figsize=(7,5))
plt.loglog(x_arr, ε_arr, label="Numerical ε(r)", lw=2)
plt.loglog(x_arr, ε_ansatz, "--", label="Analytic ansatz", lw=2)
plt.axvline(x_plus, color="k", ls=":", alpha=.4)
plt.text(1.04, 0.25, "$r_+$", transform=plt.gca().transAxes)
plt.xlabel(r"$\hat r = r/r_g$")
plt.ylabel(r"$\epsilon(r)=2\,\psi(r)$")
plt.title("Quaternionic flux in Kerr spacetime (fixed units)")
plt.grid(alpha=.3, ls="--"); plt.legend()
plt.tight_layout(); plt.show()

# ---- δc_g/c 估算 (f = 0.01 Hz) -----------------------------------
f_LISA = 1.0e-2
k      = 2*pi*f_LISA / c      # s⁻¹
idx10  = np.argmin(abs(x_arr - 10))
ψ10    = ψ_arr[idx10]
deltac = (ψ10**2 * (a_hat/x_arr[idx10])**2) / (2 * k**2 * r_g_m**2)

print(f"\nAt 10 r_g:  ε ≈ {2*ψ10:.3e}")
print(f"Predicted δc_g/c ≈ {deltac:.2e}")
